In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fifa-22-complete-player-dataset/female_players_17.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_20.csv
/kaggle/input/fifa-22-complete-player-dataset/players_15.csv
/kaggle/input/fifa-22-complete-player-dataset/players_16.csv
/kaggle/input/fifa-22-complete-player-dataset/Career Mode female player datasets - FIFA 16-22.xlsx
/kaggle/input/fifa-22-complete-player-dataset/players_17.csv
/kaggle/input/fifa-22-complete-player-dataset/players_18.csv
/kaggle/input/fifa-22-complete-player-dataset/players_19.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_16.csv
/kaggle/input/fifa-22-complete-player-dataset/players_22.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_18.csv
/kaggle/input/fifa-22-complete-player-dataset/players_21.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_19.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_22.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_21.cs

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
from plotly.offline import init_notebook_mode, iplot, plot
from plotly.subplots import make_subplots
import plotly.express as px
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.8f' % x)

In [5]:
fifa22_df = pd.read_csv('/kaggle/input/fifa-22-complete-player-dataset/players_22.csv')

In [6]:
fifa22_df.head(1)

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.00000000,320000.00000000,34,1987-06-24,170,72,73.00000000,Paris Saint-Germain,French Ligue 1,1.00000000,RW,30.00000000,NaN,2021-08-10,2023.00000000,52,Argentina,1369.00000000,RW,10.00000000,Left,4,4,5,Medium/Low,Unique,Yes,144300000.00000000,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Playmaker ...",85.00000000,92.00000000,91.00000000,95.00000000,34.00000000,65.00000000,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,20,35,24,6,11,15,14,8,NaN,89+3,89+3,89+3,92,93,93,93,92,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png


In [7]:
fifa22_filtered = fifa22_df[['sofifa_id', 'short_name', 'player_positions', 'overall', 'potential', 'club_name','value_eur',
                            'wage_eur', 'age', 'height_cm', 'weight_kg', 'preferred_foot', 'weak_foot', 'work_rate','release_clause_eur',
                            'pace','shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing',
                            'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing',
                            'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
                            'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
                            'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina',
                            'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
                            'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 'defending_marking_awareness',
                            'defending_standing_tackle', 'defending_sliding_tackle', 'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
                            'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed','ls','st','rs','lw','lf','cf',
                            'rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk']]

In [8]:
fifa22_filtered.isna().sum()

sofifa_id           0
short_name          0
player_positions    0
overall             0
potential           0
                   ..
lcb                 0
cb                  0
rcb                 0
rb                  0
gk                  0
Length: 83, dtype: int64

In [9]:
#Fill numeric na features with 0
fifa22_filtered['value_eur']          = fifa22_filtered['value_eur'].fillna(0)
fifa22_filtered['wage_eur']           = fifa22_filtered['wage_eur'].fillna(0)
fifa22_filtered['release_clause_eur'] = fifa22_filtered['release_clause_eur'].fillna(0)
fifa22_filtered['goalkeeping_speed']  = fifa22_filtered['goalkeeping_speed'].fillna(0)

In [10]:
#View the rows with null features
fifa22_filtered[fifa22_filtered['shooting'].isna()].head(10)

,sofifa_id,short_name,player_positions,overall,potential,club_name,value_eur,wage_eur,age,height_cm,weight_kg,preferred_foot,weak_foot,work_rate,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
5,200389,J. Oblak,GK,91,93,Atlético de Madrid,112000000.00000000,130000.00000000,28,188,87,Right,3,Medium/Medium,238000000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68,27,12,18,87,92,78,90,90,50.00000000,33+3,33+3,33+3,32,35,35,35,32,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3,89+3
7,167495,M. Neuer,GK,90,90,FC Bayern München,13500000.00000000,86000.00000000,35,193,93,Right,4,Medium/Medium,22300000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,60,11,30,14,11,68,46,54,60,51,87,35,68,77,43,80,16,29,30,12,70,47,70,17,10,11,88,88,91,89,88,56.00000000,40+3,40+3,40+3,40,43,43,43,40,47+3,47+3,47+3,44+3,50+3,50+3,50+3,44+3,37+3,43+3,43+3,43+3,37+3,35+3,34+3,34+3,34+3,35+3,88+2
8,192448,M. ter Stegen,GK,90,92,FC Barcelona,99000000.00000000,250000.00000000,29,187,85,Right,4,Medium/Medium,210400000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,18,14,11,61,14,21,18,12,63,30,38,50,39,86,43,66,79,35,78,10,43,22,11,70,25,70,25,13,10,88,85,88,88,90,43.00000000,35+3,35+3,35+3,35,38,38,38,35,42+3,42+3,42+3,39+3,45+3,45+3,45+3,39+3,33+3,41+3,41+3,41+3,33+3,31+3,33+3,33+3,33+3,31+3,88+3
12,192119,T. Courtois,GK,89,91,Real Madrid CF,85500000.00000000,250000.00000000,29,199,96,Left,3,Medium/Medium,181700000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,14,14,13,33,12,13,19,20,35,23,42,52,62,84,45,56,68,38,70,17,23,15,13,44,27,66,20,18,16,84,89,74,86,88,46.00000000,31+3,31+3,31+3,29,31,31,31,29,32+3,32+3,32+3,31+3,32+3,32+3,32+3,31+3,29+3,31+3,31+3,31+3,29+3,29+3,29+3,29+3,29+3,29+3,86+3
18,210257,Ederson,GK,89,91,Manchester City,94000000.00000000,200000.00000000,27,188,86,Left,3,Medium/Medium,181000000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,20,14,14,61,18,23,15,20,66,40,64,63,60,88,48,70,66,41,78,18,38,27,20,70,51,70,29,15,8,87,82,93,88,88,64.00000000,40+3,40+3,40+3,41,43,43,43,41,47+3,47+3,47+3,44+3,49+3,49+3,49+3,44+3,37+3,44+3,44+3,44+3,37+3,36+3,35+3,35+3,35+3,36+3,87+3
20,212831,Alisson,GK,89,90,Liverpool,82000000.00000000,190000.00000000,28,191,91,Right,3,Medium/Medium,157900000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,17,13,29,45,20,27,19,18,44,30,56,47,40,87,37,63,52,32,78,14,27,11,13,66,23,65,15,19,16,86,86,84,90,89,52.00000000,37+3,37+3,37+3,35,38,38,38,35,40+3,40+3,40+3,38+3,40+3,40+3,40+3,38+3,31+3,35+3,35+3,35+3,31+3,30+3,31+3,31+3,31+3,30+3,87+3
21,230621,G. Donnarumma,GK,89,93,Paris Saint-Germain,119500000.00000000,110000.00000000,22,196,90,Right,3,Medium/Medium,230000000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,12,12,12,36,8,28,12,14,34,30,50,55,64,85,38,59,72,34,72,18,30,26,14,60,24,68,20,14,16,91,83,79,85,90,52.00000000,34+3,34+3,34+3,34,36,36,36,34,38+3,38+3,38+3,35+3,37+3,37+3,37+3,35+3,31+3,34+3,34+3,34+3,31+3,31+3,31+3,31+3,31+3,31+3,87+3
26,193041,K. Navas,GK,88,88,Paris Saint-Germain,15500000.00000000,130000.00000000,34,185,80,Right,3,Medium/Medium,28700000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,11,15,11,40,11,24,21,15,37,30,54,53,60,85,61,56,74,39,75,13,32,22,16,68,25,67,28,14,14,89,84,75,87,

In [11]:
#since all the remaining null values are for GK. impute the cells with 0
fifa22_filtered = fifa22_filtered.fillna(0)
fifa22_filtered = fifa22_filtered.fillna(' ')

In [12]:
fifa22_filtered['release_clause_eur'].dtypes

dtype('float64')

In [13]:
fifa22_filtered.head(10)

,sofifa_id,short_name,player_positions,overall,potential,club_name,value_eur,wage_eur,age,height_cm,weight_kg,preferred_foot,weak_foot,work_rate,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,158023,L. Messi,"RW, ST, CF",93,93,Paris Saint-Germain,78000000.00000000,320000.00000000,34,170,72,Left,4,Medium/Low,144300000.00000000,85.00000000,92.00000000,91.00000000,95.00000000,34.00000000,65.00000000,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,20,35,24,6,11,15,14,8,0.00000000,89+3,89+3,89+3,92,93,93,93,92,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3
1,188545,R. Lewandowski,ST,92,92,FC Bayern München,119500000.00000000,270000.00000000,32,185,81,Right,4,High/Medium,197200000.00000000,78.00000000,92.00000000,79.00000000,86.00000000,44.00000000,82.00000000,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,88,35,42,19,15,6,12,8,10,0.00000000,90+2,90+2,90+2,85,88,88,88,85,86+3,86+3,86+3,84+3,80+3,80+3,80+3,84+3,64+3,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
2,20801,Cristiano Ronaldo,"ST, LW",91,91,Manchester United,45000000.00000000,270000.00000000,36,187,83,Right,4,High/Low,83300000.00000000,87.00000000,94.00000000,80.00000000,88.00000000,34.00000000,75.00000000,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,95,24,32,24,7,11,15,14,11,0.00000000,90+1,90+1,90+1,88,89,89,89,88,86+3,86+3,86+3,86+3,78+3,78+3,78+3,86+3,63+3,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3
3,190871,Neymar Jr,"LW, CAM",91,91,Paris Saint-Germain,129000000.00000000,270000.00000000,29,175,68,Right,5,High/Medium,238700000.00000000,91.00000000,83.00000000,86.00000000,94.00000000,37.00000000,63.00000000,85,83,63,86,86,95,88,87,81,95,93,89,96,89,84,80,64,81,53,81,63,37,86,90,93,93,35,32,29,9,9,15,15,11,0.00000000,83+3,83+3,83+3,90,88,88,88,90,89+2,89+2,89+2,89+2,82+3,82+3,82+3,89+2,67+3,63+3,63+3,63+3,67+3,62+3,50+3,50+3,50+3,62+3,20+3
4,192985,K. De Bruyne,"CM, CAM",91,91,Manchester City,125500000.00000000,350000.00000000,30,181,70,Right,5,High/High,232200000.00000000,76.00000000,86.00000000,93.00000000,88.00000000,64.00000000,78.00000000,94,82,55,94,82,88,85,83,93,91,76,76,79,91,78,91,63,89,74,91,76,66,88,94,83,89,68,65,53,15,13,5,10,13,0.00000000,83+3,83+3,83+3,88,87,87,87,88,89+2,89+2,89+2,89+2,89+2,89+2,89+2,89+2,79+3,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3
5,200389,J. Oblak,GK,91,93,Atlético de Madrid,112000000.00000000,130000.00000000,28,188,87,Right,3,Medium/Medium,238000000.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68,27,12,18,87,92,78,90,90,50.00000000,33+3,33+3,33+3,32,35,35,35,32,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3,89+3
6,231747,K. Mbappé,"ST, LW",91,95,Paris Saint-Germain,194000000.00000000,230000.00000000,22,182,73,Right,4,High/Low,373500000.00000000,97.00000000,88.00000000,80.00000000,92.00000000,36.00000000,77.00000000,78,93,72,85,83,93,80,69,71,91,97,97,92,93,83,86,78,88,77,82,62,38,92,82,79,88,26,34,32,13,5,7,11,6,0.00000000,89+3,89+3,89+3,90,90,90,90

In [14]:
fifa22_filtered[['atk_workrate','def_workrate']] = fifa22_filtered['work_rate'].str.split('/',expand=True)

In [15]:
fifa22_filtered.drop(['work_rate'], axis=1, inplace=True)

In [16]:
fifa22_filtered['sofifa_id'].nunique()

19239

In [17]:
fifa22_filtered['ls'] = fifa22_filtered.ls.str.split('+').str[0]
fifa22_filtered['st'] = fifa22_filtered['st'].str.split('+').str[0]
fifa22_filtered['rs'] = fifa22_filtered['rs'].str.split('+').str[0]
fifa22_filtered['lw'] = fifa22_filtered['lw'].str.split('+').str[0]
fifa22_filtered['lf'] = fifa22_filtered['lf'].str.split('+').str[0]
fifa22_filtered['cf'] = fifa22_filtered['cf'].str.split('+').str[0]
fifa22_filtered['rf'] = fifa22_filtered['rf'].str.split('+').str[0]
fifa22_filtered['rw'] = fifa22_filtered['rw'].str.split('+').str[0]
fifa22_filtered['lam'] = fifa22_filtered['lam'].str.split('+').str[0]
fifa22_filtered['cam'] = fifa22_filtered['cam'].str.split('+').str[0]
fifa22_filtered['ram'] = fifa22_filtered['ram'].str.split('+').str[0]
fifa22_filtered['lm'] = fifa22_filtered['lm'].str.split('+').str[0]
fifa22_filtered['lcm'] = fifa22_filtered['lcm'].str.split('+').str[0]
fifa22_filtered['cm'] = fifa22_filtered['cm'].str.split('+').str[0]
fifa22_filtered['rcm'] = fifa22_filtered['rcm'].str.split('+').str[0]
fifa22_filtered['rm'] = fifa22_filtered['rm'].str.split('+').str[0]
fifa22_filtered['lwb'] = fifa22_filtered['lwb'].str.split('+').str[0]
fifa22_filtered['ldm'] = fifa22_filtered['ldm'].str.split('+').str[0]
fifa22_filtered['cdm'] = fifa22_filtered['cdm'].str.split('+').str[0]
fifa22_filtered['rdm'] = fifa22_filtered['rdm'].str.split('+').str[0]
fifa22_filtered['rwb'] = fifa22_filtered['rwb'].str.split('+').str[0]
fifa22_filtered['lb'] = fifa22_filtered['lb'].str.split('+').str[0]
fifa22_filtered['lcb'] = fifa22_filtered['lcb'].str.split('+').str[0]
fifa22_filtered['cb'] = fifa22_filtered['cb'].str.split('+').str[0]
fifa22_filtered['rcb'] = fifa22_filtered['rcb'].str.split('+').str[0]
fifa22_filtered['rb'] = fifa22_filtered['rb'].str.split('+').str[0]
fifa22_filtered['gk'] = fifa22_filtered['gk'].str.split('+').str[0]

In [18]:
fifa22_filtered['ls'] = fifa22_filtered.ls.str.split('-').str[0]
fifa22_filtered['st'] = fifa22_filtered['st'].str.split('-').str[0]
fifa22_filtered['rs'] = fifa22_filtered['rs'].str.split('-').str[0]
fifa22_filtered['lw'] = fifa22_filtered['lw'].str.split('-').str[0]
fifa22_filtered['lf'] = fifa22_filtered['lf'].str.split('-').str[0]
fifa22_filtered['cf'] = fifa22_filtered['cf'].str.split('-').str[0]
fifa22_filtered['rf'] = fifa22_filtered['rf'].str.split('-').str[0]
fifa22_filtered['rw'] = fifa22_filtered['rw'].str.split('-').str[0]
fifa22_filtered['lam'] = fifa22_filtered['lam'].str.split('-').str[0]
fifa22_filtered['cam'] = fifa22_filtered['cam'].str.split('-').str[0]
fifa22_filtered['ram'] = fifa22_filtered['ram'].str.split('-').str[0]
fifa22_filtered['lm'] = fifa22_filtered['lm'].str.split('-').str[0]
fifa22_filtered['lcm'] = fifa22_filtered['lcm'].str.split('-').str[0]
fifa22_filtered['cm'] = fifa22_filtered['cm'].str.split('-').str[0]
fifa22_filtered['rcm'] = fifa22_filtered['rcm'].str.split('-').str[0]
fifa22_filtered['rm'] = fifa22_filtered['rm'].str.split('-').str[0]
fifa22_filtered['lwb'] = fifa22_filtered['lwb'].str.split('-').str[0]
fifa22_filtered['ldm'] = fifa22_filtered['ldm'].str.split('-').str[0]
fifa22_filtered['cdm'] = fifa22_filtered['cdm'].str.split('-').str[0]
fifa22_filtered['rdm'] = fifa22_filtered['rdm'].str.split('-').str[0]
fifa22_filtered['rwb'] = fifa22_filtered['rwb'].str.split('-').str[0]
fifa22_filtered['lb'] = fifa22_filtered['lb'].str.split('-').str[0]
fifa22_filtered['lcb'] = fifa22_filtered['lcb'].str.split('-').str[0]
fifa22_filtered['cb'] = fifa22_filtered['cb'].str.split('-').str[0]
fifa22_filtered['rcb'] = fifa22_filtered['rcb'].str.split('-').str[0]
fifa22_filtered['rb'] = fifa22_filtered['rb'].str.split('-').str[0]
fifa22_filtered['gk'] = fifa22_filtered['gk'].str.split('-').str[0]

In [19]:
fifa22_filtered['ls'] = fifa22_filtered['ls'].astype(int)
fifa22_filtered['st'] = fifa22_filtered['st'].astype(int)
fifa22_filtered['rs'] = fifa22_filtered['rs'].astype(int)
fifa22_filtered['lw'] = fifa22_filtered['lw'].astype(int)
fifa22_filtered['lf'] = fifa22_filtered['lf'].astype(int)
fifa22_filtered['cf'] = fifa22_filtered['cf'].astype(int)
fifa22_filtered['rf'] = fifa22_filtered['rf'].astype(int)
fifa22_filtered['rw'] = fifa22_filtered['rw'].astype(int)
fifa22_filtered['lam'] = fifa22_filtered['lam'].astype(int)
fifa22_filtered['cam'] = fifa22_filtered['cam'].astype(int)
fifa22_filtered['ram'] = fifa22_filtered['ram'].astype(int)
fifa22_filtered['lm'] = fifa22_filtered['lm'].astype(int)
fifa22_filtered['lcm'] = fifa22_filtered['lcm'].astype(int)
fifa22_filtered['cm'] = fifa22_filtered['cm'].astype(int)
fifa22_filtered['rcm'] = fifa22_filtered['rcm'].astype(int)
fifa22_filtered['rm'] = fifa22_filtered['rm'].astype(int)
fifa22_filtered['lwb'] = fifa22_filtered['lwb'].astype(int)
fifa22_filtered['ldm'] = fifa22_filtered['ldm'].astype(int)
fifa22_filtered['cdm'] = fifa22_filtered['cdm'].astype(int)
fifa22_filtered['rdm'] = fifa22_filtered['rdm'].astype(int)
fifa22_filtered['rwb'] = fifa22_filtered['rwb'].astype(int)
fifa22_filtered['lb'] = fifa22_filtered['lb'].astype(int)
fifa22_filtered['lcb'] = fifa22_filtered['lcb'].astype(int)
fifa22_filtered['cb'] = fifa22_filtered['cb'].astype(int)
fifa22_filtered['rcb'] = fifa22_filtered['rcb'].astype(int)
fifa22_filtered['rb'] = fifa22_filtered['rb'].astype(int)
fifa22_filtered['gk'] = fifa22_filtered['gk'].astype(int)

In [20]:
fifa22_filtered = pd.get_dummies(fifa22_filtered, columns=['atk_workrate','def_workrate'])

In [21]:
fifa22_filtered['preferred_foot'] = pd.factorize(fifa22_filtered['preferred_foot'])[0]

In [22]:
fifa22_filtered[fifa22_filtered['short_name'] == 'E. Hazard']


,sofifa_id,short_name,player_positions,overall,potential,club_name,value_eur,wage_eur,age,height_cm,weight_kg,preferred_foot,weak_foot,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,atk_workrate_High,atk_workrate_Low,atk_workrate_Medium,def_workrate_High,def_workrate_Low,def_workrate_Medium
75,183277,E. Hazard,LW,85,85,Real Madrid CF,52000000.00000000,240000.00000000,30,175,74,1,4,106600000.00000000,84.00000000,80.00000000,82.00000000,88.00000000,35.00000000,63.00000000,77,78,61,85,82,89,83,79,81,88,86,82,87,79,89,82,55,77,61,77,54,41,83,84,87,89,34,27,22,11,12,6,8,8,0.00000000,79,79,79,84,83,83,83,84,84,84,84,83,78,78,78,83,63,61,61,61,63,58,48,48,48,58,17,0,0,1,0,0,1


In [23]:
feature_cols = ['overall', 'potential', 'pace','shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing','attacking_finishing', 
                'attacking_heading_accuracy', 'attacking_short_passing','attacking_volleys', 'skill_dribbling', 'skill_curve', 
                'skill_fk_accuracy', 'skill_long_passing','skill_ball_control','movement_acceleration','movement_sprint_speed', 
                'movement_agility','movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina',
                'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
                'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 'defending_marking_awareness',
                'defending_standing_tackle', 'defending_sliding_tackle', 'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
                'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed', 'preferred_foot','weak_foot',
                'age', 'height_cm', 'weight_kg','ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm',
                'cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk']

In [24]:
fifa22_filtered[feature_cols].head()

,overall,potential,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,preferred_foot,weak_foot,age,height_cm,weight_kg,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,93,93,85.00000000,92.00000000,91.00000000,95.00000000,34.00000000,65.00000000,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,20,35,24,6,11,15,14,8,0.00000000,0,4,34,170,72,89,89,89,92,93,93,93,92,93,93,93,91,87,87,87,91,66,64,64,64,66,61,50,50,50,61,19
1,92,92,78.00000000,92.00000000,79.00000000,86.00000000,44.00000000,82.00000000,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,88,35,42,19,15,6,12,8,10,0.00000000,1,4,32,185,81,90,90,90,85,88,88,88,85,86,86,86,84,80,80,80,84,64,66,66,66,64,61,60,60,60,61,19
2,91,91,87.00000000,94.00000000,80.00000000,88.00000000,34.00000000,75.00000000,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,95,24,32,24,7,11,15,14,11,0.00000000,1,4,36,187,83,90,90,90,88,89,89,89,88,86,86,86,86,78,78,78,86,63,59,59,59,63,60,53,53,53,60,20
3,91,91,91.00000000,83.00000000,86.00000000,94.00000000,37.00000000,63.00000000,85,83,63,86,86,95,88,87,81,95,93,89,96,89,84,80,64,81,53,81,63,37,86,90,93,93,35,32,29,9,9,15,15,11,0.00000000,1,5,29,175,68,83,83,83,90,88,88,88,90,89,89,89,89,82,82,82,89,67,63,63,63,67,62,50,50,50,62,20
4,91,91,76.00000000,86.00000000,93.00000000,88.00000000,64.00000000,78.00000000,94,82,55,94,82,88,85,83,93,91,76,76,79,91,78,91,63,89,74,91,76,66,88,94,83,89,68,65,53,15,13,5,10,13,0.00000000,1,5,30,181,70,83,83,83,88,87,87,87,88,89,89,89,89,89,89,89,89,79,80,80,80,79,75,69,69,69,75,21


In [25]:
train   = fifa22_filtered[['sofifa_id','short_name']+feature_cols].dropna()
id_name = fifa22_filtered[['sofifa_id','short_name']].set_index('sofifa_id')['short_name'].to_dict()

In [26]:
def normalize(df):
    scaler = MinMaxScaler()
    #result = pd.DataFrame(scaler.fit_transform(df.to_numpy()), columns=df.columns, index=df.index)
    result = df.copy()
    for feature_name in df.columns:
        max_value = float(df[feature_name].max())
        min_value = float(df[feature_name].min())
        result[feature_name] = ((result[feature_name].apply(lambda x: float(x)) - min_value)/(max_value - min_value)).astype(float) * 100
        #result[feature_name] = scaler.fit_transform(result[feature_name])
    
    return result

In [27]:
K_recomendations = 10

X = train[feature_cols]
nbrs = NearestNeighbors(n_neighbors=K_recomendations+1, metric='cosine', algorithm='brute').fit(X)
dist, rank = nbrs.kneighbors(X)

similar_df = pd.DataFrame(columns=[f'rank_{i}'for i in range(1,K_recomendations+1)],
                          index=train['sofifa_id'].values,
                          data=rank[:,1:])
dist_df = pd.DataFrame(columns=[f'rank_{i}'for i in range(1,K_recomendations+1)],
                       index=train['sofifa_id'].values,
                       data=dist[:,1:])


for cols in list(similar_df):
    tg_col = similar_df[cols]
    new_value = train['sofifa_id'].iloc[tg_col].tolist()
    similar_df[cols] = new_value

In [28]:
dist_df.head()

,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
158023,0.00253661,0.00274304,0.00287449,0.00310596,0.00330284,0.00373568,0.00376501,0.00378676,0.00398483,0.00426353
188545,0.00127756,0.00177004,0.00179090,0.00214130,0.00222850,0.00223872,0.00225667,0.00243310,0.00244935,0.00254841
20801,0.00207207,0.00227052,0.00234413,0.00243304,0.00251519,0.00255263,0.00262860,0.00263271,0.00264146,0.00265370
190871,0.00110773,0.00182431,0.00208395,0.00227793,0.00233528,0.00234129,0.00238428,0.00245171,0.00250811,0.00253910
192985,0.00112934,0.00145523,0.00179889,0.00196978,0.00198665,0.00204876,0.00216789,0.00221351,0.00228992,0.00229943


In [29]:
def similar_player(similar_df, dist_df, player_id):
    player_id = int(player_id)
    player_name = train[train['sofifa_id'] == player_id]['short_name'].values[0]
    
    ## Bar chart
    Xaxis = dist_df.loc[player_id].values[::-1]
    Yaxis = similar_df.loc[player_id].map(id_name).values[::-1]

    fig = go.Figure(go.Bar(
                x=Xaxis,
                y=Yaxis,
                orientation='h'))
    
    fig.update_layout(title_text='Players similar to  " '+str(player_name)+' "')
    fig.show()

    ## Table display
    sim_ply = similar_df.loc[player_id]
    display_col = ['short_name','age','overall','potential', 'preferred_foot','weak_foot','player_positions','work_rate','club_name',
                   'value_eur','wage_eur','release_clause_eur']

    display_df = pd.DataFrame({'sofifa_id':similar_df.loc[player_id]})\
    .merge(fifa22_df[['sofifa_id']+display_col], how='left', on='sofifa_id')[display_col]


    display(display_df)

In [30]:
similar_player(similar_df, dist_df, '158023')

,short_name,age,overall,potential,preferred_foot,weak_foot,player_positions,work_rate,club_name,value_eur,wage_eur,release_clause_eur
0,P. Dybala,27,87,88,Left,3,"CF, CAM",Medium/Medium,Juventus,93000000.00000000,160000.00000000,160400000.00000000
1,E. Hazard,30,85,85,Right,4,LW,Medium/Medium,Real Madrid CF,52000000.00000000,240000.00000000,106600000.00000000
2,Neymar Jr,29,91,91,Right,5,"LW, CAM",High/Medium,Paris Saint-Germain,129000000.00000000,270000.00000000,238700000.00000000
3,C. Vela,32,83,83,Left,2,"RW, LW, CAM",High/Low,Los Angeles FC,29000000.00000000,17000.00000000,43500000.00000000
4,R. Mahrez,30,86,86,Left,4,"RW, RM",Medium/Medium,Manchester City,65500000.00000000,220000.00000000,121200000.00000000
5,Claudio Coíntra,25,77,77,Left,5,"CAM, CF",High/Medium,Palmeiras,11500000.00000000,36000.00000000,21900000.00000000
6,W. Ben Yedder,30,84,84,Right,5,ST,High/Low,AS Monaco,41500000.00000000,88000.00000000,78900000.00000000
7,Iago Aspas,33,84,84,Left,4,"ST, RW",High/High,RC Celta de Vigo,27000000.00000000,46000.00000000,56700000.00000000
8,A. Miranchuk,25,78,80,Left,2,"CAM, ST",High/Medium,Atalanta,17500000.00000000,46000.00000000,31100000.00000000
9,M. Depay,27,85,86,Right,3,"CF, LW, CAM",High/Low,FC Barcelona,63000000.00000000,220000.00000000,133900000.00000000


In [31]:
fifa22_df[fifa22_df['long_name'].str.contains('Messi')]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.00000000,320000.00000000,34,1987-06-24,170,72,73.00000000,Paris Saint-Germain,French Ligue 1,1.00000000,RW,30.00000000,NaN,2021-08-10,2023.00000000,52,Argentina,1369.00000000,RW,10.00000000,Left,4,4,5,Medium/Low,Unique,Yes,144300000.00000000,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Playmaker ...",85.00000000,92.00000000,91.00000000,95.00000000,34.00000000,65.00000000,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,20,35,24,6,11,15,14,8,NaN,89+3,89+3,89+3,92,93,93,93,92,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
2368,240938,https://sofifa.com/player/240938/junior-messia...,Junior Messias,Junior Walter Messias,"CAM, CF, CM",74,74,4100000.00000000,12000.00000000,30,1991-05-13,179,70,47.00000000,AC Milan,Italian Serie A,1.00000000,SUB,30.00000000,Crotone,NaN,2022.00000000,54,Brazil,NaN,NaN,NaN,Left,3,3,1,Medium/Medium,Lean (170-185),No,NaN,NaN,"Speed Dribbler (AI), Team Player",82.00000000,68.00000000,72.00000000,76.00000000,37.00000000,54.00000000,66,69,52,76,61,79,72,73,69,76,80,83,75,69,68,68,53,65,52,70,45,44,70,71,62,61,31,35,27,8,8,15,9,12,NaN,69+2,69+2,69+2,74,73,73,73,74,74,74,74,74,69+2,69+2,69+2,74,58+2,56+2,56+2,56+2,58+2,55+2,46+2,46+2,46+2,55+2,16+2,https://cdn.sofifa.net/players/240/938/22_120.png,https://cdn.sofifa.net/teams/47/60.png,https://cdn.sofifa.net/flags/it.png,NaN,https://cdn.sofifa.net/flags/br.png
5645,202350,https://sofifa.com/player/202350/fabricio-dos-...,Fabrício,Fabrício dos Santos Messias,"ST, LW",69,69,1200000.00000000,5000.00000000,31,1990-03-28,183,77,10031.00000000,Portimonense SC,Portuguese Liga ZON SAGRES,1.00000000,ST,9.00000000,NaN,2021-07-01,2024.00000000,54,Brazil,NaN,NaN,NaN,Right,2,4,1,Medium/Medium,Normal (170-185),No,2500000.00000000,NaN,NaN,64.00000000,70.00000000,66.00000000,67.00000000,33.00000000,75.00000000,64,68,69,67,77,66,70,70,61,67,65,64,73,67,54,77,81,69,79,66,73,36,71,69,74,74,25,27,30,9,14,13,10,10,NaN,69,69,69,67,68,68,68,67,67+2,67+2,67+2,67+2,63+2,63+2,63+2,67+2,53+2,53+2,53+2,53+2,53+2,50+2,49+2,49+2,49+2,50+2,17+2,https://cdn.sofifa.net/players/202/350/22_120.png,https://cdn.sofifa.net/teams/10031/60.png,https://cdn.sofifa.net/flags/pt.png,NaN,https://cdn.so

In [32]:
fifa22_chelsea = fifa22_filtered[fifa22_filtered['club_name'] == 'Chelsea']

In [33]:
fifa22_chelsea

,sofifa_id,short_name,player_positions,overall,potential,club_name,value_eur,wage_eur,age,height_cm,weight_kg,preferred_foot,weak_foot,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,atk_workrate_High,atk_workrate_Low,atk_workrate_Medium,def_workrate_High,def_workrate_Low,def_workrate_Medium
10,215914,N. Kanté,"CDM, CM",90,90,Chelsea,100000000.00000000,230000.00000000,30,168,70,1,3,185000000.00000000,78.00000000,66.00000000,75.00000000,82.00000000,87.00000000,83.00000000,68,65,54,82,56,79,49,49,79,81,82,75,82,93,92,71,77,97,72,65,93,91,72,78,54,84,90,93,86,15,12,10,7,10,0.00000000,72,72,72,77,77,77,77,77,78,78,78,79,82,82,82,79,85,87,87,87,85,85,84,84,84,85,20,0,0,1,1,0,0
25,192505,R. Lukaku,ST,88,88,Chelsea,93500000.00000000,260000.00000000,28,191,94,0,4,173000000.00000000,84.00000000,87.00000000,74.00000000,78.00000000,39.00000000,83.00000000,73,92,89,77,78,83,78,66,70,77,81,87,60,91,52,89,76,71,95,74,71,27,89,75,88,86,30,41,30,8,15,14,7,10,0.00000000,87,87,87,81,84,84,84,81,80,80,80,80,74,74,74,80,61,59,59,59,61,58,57,57,57,58,19,0,0,1,0,0,1
71,164240,Thiago Silva,CB,85,85,Chelsea,9500000.00000000,105000.00000000,36,183,79,1,3,17600000.00000000,53.00000000,54.00000000,72.00000000,72.00000000,86.00000000,78.00000000,60,40,81,80,61,67,62,61,80,79,52,54,67,82,66,71,88,68,82,65,76,88,59,71,60,86,87,86,84,9,12,5,9,10,0.00000000,65,65,65,65,67,67,67,65,69,69,69,67,75,75,75,67,76,82,82,82,76,77,83,83,83,77,18,0,0,1,1,0,0
88,205498,Jorginho,"CM, CDM",85,85,Chelsea,52500000.00000000,170000.00000000,29,180,68,1,3,97100000.00000000,55.00000000,67.00000000,86.00000000,81.00000000,73.00000000,73.00000000,75,64,56,92,71,79,75,76,89,87,61,50,77,82,72,71,58,87,66,62,79,81,69,88,91,81,67,81,71,14,7,13,14,8,0.00000000,70,70,70,76,76,76,76,76,80,80,80,78,84,84,84,78,78,82,82,82,78,76,73,73,73,76,19,0,0,1,0,0,1
123,208670,H. Ziyech,"RW, CAM",84,84,Chelsea,42500000.00000000,170000.00000000,28,181,65,0,2,78600000.00000000,78.00000000,76.00000000,87.00000000,83.00000000,50.00000000,65.00000000,88,73,52,86,74,83,90,78,89,85,79,77,85,84,78,83,35,79,58,77,73,52,82,89,67,79,44,56,40,11,7,11,8,14,0.00000000,76,76,76,83,82,82,82,83,84,84,84,83,82,82,82,83,70,70,70,70,70,66,57,57,57,66,18,1,0,0,1,0,0
126,212188,T. Werner,"ST, LW",84,87,Chelsea,56500000.00000000,155000.00000000,25,180,76,1,4,108800000.00000000,91.00000000,81.00000000,70.00000000,82.00000000,35.00000000,70.00000000,69,81,76,79,84,82,70,58,55,84,93,89,82,87,76,85,75,83,65,77,63,37,88,68,78,79,39,20,19,9,9,8,15,9,0.00000000,83,83,83,82,83,83,83,82,80,80,80,80,72,72,72,80,61,57,57,57,61,58,50,50,50,58,19,1,0,0,0,0,1
138,235790,K. Havertz,"CAM, CF, CM",84,92,Chelsea,94500000.00000000,130000.00000000,22,188,82,0,4,193700000.00000000,82.00000000,78.00000000,79.00000000,84.00000000,45.00000000,66.00000000,72,81,78,84,80,86,84,53,78,85,77,86,78,83,75,74,75,77,69,77,42,38,82,81,73,84,39,46,38,6,8,11,10,12,0.00000000,81,81,81,82,82,82,82,82,83,83,83,82,78,78,78,82,65,63,63,63,65,62,55,55,55,62,17,0,0,1,0,0,1
148,184432,Azpilicueta,"CB, RWB",83,83,Chelsea,25000000.00000000,125000.00000000,31,178,77,1,3,46300000.00000000,66.00000000,55.00000000,76.00000000,72.00000000,84.00000000,76.00000000,80,46,77,79,51,68,6

In [34]:
feature_clubcols = ['overall', 'potential', 'pace','shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing','attacking_finishing', 
                'attacking_heading_accuracy', 'attacking_short_passing','attacking_volleys', 'skill_dribbling', 'skill_curve', 
                'skill_fk_accuracy', 'skill_long_passing','skill_ball_control','movement_acceleration','movement_sprint_speed', 
                'movement_agility','movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina',
                'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
                'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 'defending_marking_awareness',
                'defending_standing_tackle', 'defending_sliding_tackle', 'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
                'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed','preferred_foot','weak_foot',
                'age', 'height_cm', 'weight_kg']

In [35]:
fifa22_chelsea = fifa22_chelsea[['sofifa_id','short_name','player_positions']+feature_clubcols].reset_index()

In [36]:
fifa22_chelsea

,index,sofifa_id,short_name,player_positions,overall,potential,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,preferred_foot,weak_foot,age,height_cm,weight_kg
0,10,215914,N. Kanté,"CDM, CM",90,90,78.00000000,66.00000000,75.00000000,82.00000000,87.00000000,83.00000000,68,65,54,82,56,79,49,49,79,81,82,75,82,93,92,71,77,97,72,65,93,91,72,78,54,84,90,93,86,15,12,10,7,10,0.00000000,1,3,30,168,70
1,25,192505,R. Lukaku,ST,88,88,84.00000000,87.00000000,74.00000000,78.00000000,39.00000000,83.00000000,73,92,89,77,78,83,78,66,70,77,81,87,60,91,52,89,76,71,95,74,71,27,89,75,88,86,30,41,30,8,15,14,7,10,0.00000000,0,4,28,191,94
2,71,164240,Thiago Silva,CB,85,85,53.00000000,54.00000000,72.00000000,72.00000000,86.00000000,78.00000000,60,40,81,80,61,67,62,61,80,79,52,54,67,82,66,71,88,68,82,65,76,88,59,71,60,86,87,86,84,9,12,5,9,10,0.00000000,1,3,36,183,79
3,88,205498,Jorginho,"CM, CDM",85,85,55.00000000,67.00000000,86.00000000,81.00000000,73.00000000,73.00000000,75,64,56,92,71,79,75,76,89,87,61,50,77,82,72,71,58,87,66,62,79,81,69,88,91,81,67,81,71,14,7,13,14,8,0.00000000,1,3,29,180,68
4,123,208670,H. Ziyech,"RW, CAM",84,84,78.00000000,76.00000000,87.00000000,83.00000000,50.00000000,65.00000000,88,73,52,86,74,83,90,78,89,85,79,77,85,84,78,83,35,79,58,77,73,52,82,89,67,79,44,56,40,11,7,11,8,14,0.00000000,0,2,28,181,65
5,126,212188,T. Werner,"ST, LW",84,87,91.00000000,81.00000000,70.00000000,82.00000000,35.00000000,70.00000000,69,81,76,79,84,82,70,58,55,84,93,89,82,87,76,85,75,83,65,77,63,37,88,68,78,79,39,20,19,9,9,8,15,9,0.00000000,1,4,25,180,76
6,138,235790,K. Havertz,"CAM, CF, CM",84,92,82.00000000,78.00000000,79.00000000,84.00000000,45.00000000,66.00000000,72,81,78,84,80,86,84,53,78,85,77,86,78,83,75,74,75,77,69,77,42,38,82,81,73,84,39,46,38,6,8,11,10,12,0.00000000,0,4,22,188,82
7,148,184432,Azpilicueta,"CB, RWB",83,83,66.00000000,55.00000000,76.00000000,72.00000000,84.00000000,76.00000000,80,46,77,79,51,68,68,58,80,77,69,63,67,86,73,68,83,77,70,60,87,84,62,72,66,78,85,85,84,13,9,10,6,5,0.00000000,1,3,31,178,77
8,170,205452,A. Rüdiger,CB,83,84,75.00000000,43.00000000,68.00000000,64.00000000,84.00000000,84.00000000,68,32,81,76,34,60,35,56,73,69,64,84,56,82,50,72,84,69,88,44,92,81,39,59,40,72,85,86,82,14,15,10,12,8,0.00000000,1,3,28,190,85
9,172,207410,M. Kovačić,"CM, CDM",83,84,70.00000000,69.00000000,83.00000000,88.00000000,69.00000000,70.00000000,73,61,42,90,76,89,79,70,82,89,77,65,87,81,84,79,75,79,63,75,77,74,72,84,59,81,70,73,66,9,5,6,7,7,0.00000000,1,4,27,176,80


In [37]:
fifa22_chelsea_norm = normalize(fifa22_chelsea[feature_clubcols])

In [38]:
fifa22_chelsea_norm.shape

(33, 48)

In [39]:
from sklearn.decomposition import PCA

In [40]:
pca = PCA(n_components=3)
reduced_features = pd.DataFrame(pca.fit_transform(fifa22_chelsea_norm))

In [41]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
kmeans = kmeans.fit(reduced_features)
labels = kmeans.predict(reduced_features)
centroid = kmeans.cluster_centers_
clusters = kmeans.labels_.tolist()

reduced_features['cluster'] = clusters
reduced_features['name']    = fifa22_chelsea['short_name'].copy()
reduced_features.columns    = ['X','Y', 'Z','cluster', 'name']

In [42]:
reduced_features

,X,Y,Z,cluster,name
0,-121.65591040,-44.46834187,-60.38853909,0,N. Kanté
1,-98.08082830,87.63388944,-38.42606874,4,R. Lukaku
2,-72.11642571,-70.73737913,-79.85952361,3,Thiago Silva
3,-104.09483645,-2.10281725,-38.17376474,0,Jorginho
4,-112.81103972,80.90600077,18.72437390,4,H. Ziyech
5,-84.92442770,102.19175282,6.37502151,4,T. Werner
6,-101.18688020,89.97605085,1.41412427,4,K. Havertz
7,-84.48194982,-63.45675111,-50.75826584,3,Azpilicueta
8,-29.78032013,-105.07988920,-63.21275510,3,A. Rüdiger
9,-116.11434454,17.20424426,-25.80658686,0,M. Kovačić


In [43]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [44]:
PLOT = go.Figure()

for C in list(reduced_features.cluster.unique()):
    
    PLOT.add_trace(go.Scatter3d(x = reduced_features[reduced_features.cluster == C]['X'],
                                y = reduced_features[reduced_features.cluster == C]['Y'],
                                z = reduced_features[reduced_features.cluster == C]['Z'],
                                mode = 'text+markers', marker_size = 6, marker_line_width = 1,
                                name = 'Cluster' + str(C), text = reduced_features[reduced_features.cluster == C]['name'],
                               textposition='top center'))

#PLOT.(textposition='top center')    


PLOT.update_layout(width = 1300, height = 1000, autosize = True, showlegend = True,
                   scene = dict(xaxis=dict(title = 'X', titlefont_color = 'black'),
                                yaxis=dict(title = 'Y', titlefont_color = 'black'),
                                zaxis=dict(title = 'Z', titlefont_color = 'black')),
                   font = dict(family = "Gilroy", color  = 'black', size = 12))